## 用户以往打分&最近打分

### 导入数据

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import multiprocessing

In [ ]:
# 导入电影数据
movies = pd.read_csv("../movies_metadata.csv")
movies = movies[['id','title','genres']]
movies.head()

In [3]:
# 整理电影类型列（将json格式转换成list）
import re
pattern = re.compile(r"e': '(.+?)'}")   # 查找数字
movies['genres'] = movies['genres'].map(lambda x: pattern.findall(x))
movies.head()

,id,title,genres
0,862,Toy Story,"[Animation, Comedy, Family]"
1,8844,Jumanji,"[Adventure, Fantasy, Family]"
2,15602,Grumpier Old Men,"[Romance, Comedy]"
3,31357,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,11862,Father of the Bride Part II,[Comedy]


rating_small

In [4]:
# 导入观众打分数据 # 这里使用rating_small
ratings = pd.read_csv('../ratings.csv')
userId_set= list(set(ratings.userId))
print('用户数量：',len(userId_set))
ratings.head()

用户数量： 270896


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


* 一个用户的全部打分数据
* 时间戳数值最大的打分记录（用户id 时间戳 电影id 分值）
* 其他打分记录计算每一类电影的平均打分

In [5]:
genres=list(np.load('genres.npy',allow_pickle=True))
#genres

In [6]:
# 自定义函数 返回一个用户的最新打分记录和以往不同类型电影平均打分
def user_genre_hist_new(userId): 
    user_ratings = ratings[ratings['userId']==userId] # 筛选出一个用户的所有打分记录
    rating_newest_id = user_ratings.timestamp.idxmax() # 最新打分的index
    rating_newest = user_ratings.loc[rating_newest_id] # 提取
#     if len(user_ratings)<6: # 若只有1条打分
#         return rating_newest.to_list()+[None]*20
    user_ratings_hist = user_ratings.drop(index = rating_newest_id) # 删除
    user_ratings_hist.index=range(len(user_ratings_hist)) # 重新设置index
    temp = [] # 用于存放某个用户所打分的电影类型标记
    for movieId in user_ratings_hist['movieId']: 
        movie_genres = movies[movies['id']==str(movieId)]['genres'].sum()
        if movie_genres==0: movie_genres=[]
        templist = [1 if genre in movie_genres else 0 for genre in genres] # 标记某个电影属于哪一类（1/0）
        temp.append(templist)

    temp_df = pd.DataFrame(temp)
    user_ratings_genres = pd.concat([user_ratings_hist,temp_df],ignore_index=True,axis=1)
    rating_list=[userId]
    for i in range(4,24):
        freq = sum(user_ratings_genres[i]) # 该用户某类电影的打分频数
        rating = None
        if(freq!=0):
            rating=sum(user_ratings_genres[2]*user_ratings_genres[i])/freq # 该用户某类电影的平均打分
        rating_list.append(rating)
    re = rating_newest.to_list()+rating_list
    print(re)
    return re

In [7]:
import multiprocessing

In [7]:
#userId_set1 = userId_set[:10000] # 10000个用户的数据
userId_set1 = userId_set # 所有用户的数据

In [13]:
# 并行计算 不要运行！！！
p = multiprocessing.Pool(10)
temp_re=p.map(user_genre_hist_new, userId_set1)
p.close()
p.join()

[2251.0, 1888.0, 4.0, 952953692.0, 2251, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[1001.0, 329.0, 4.0, 839659643.0, 1001, None, None, None, 4.0, 3.75, None, None, 4.0, 4.0, None, 4.666666666666667, None, 4.0, 4.142857142857143, None, 4.5, 5.0, 4.333333333333333, None, None]
[501.0, 631.0, 3.0, 852894781.0, 501, None, 4.5, 2.0, 5.0, 4.0, None, None, 3.0, 4.5, None, 3.4, None, 4.5, 3.5555555555555554, None, 4.25, 3.0, 4.5, None, 3.0]
[2252.0, 2316.0, 2.5, 1392081282.0, 2252, None, None, None, 4.0, None, None, None, None, None, None, None, None, 0.5, 2.5, None, 0.5, None, None, None, 1.75]
[1.0, 73017.0, 5.0, 1425942699.0, 1, None, None, None, 4.5, 4.25, None, None, None, 3.5, None, 1.0, None, None, 4.125, 3.5, None, None, None, 5.0, 3.3333333333333335]
[2253.0, 1196.0, 3.5, 1456367462.0, 2253, None, None, None, None, None, None, None, None, 3.5, None, 3.5, None, None, None, None, 3.5, None, None, None, None]
[

In [15]:
user_hist_new_df = pd.DataFrame(temp_re)

In [20]:
user_hist_new_df_fill0 = user_hist_new_df.fillna(value=0)

In [21]:
np.save('user_hist_new_df_fill0',user_hist_new_df_fill0)

In [4]:
user_hist_new_df_fill0 = pd.DataFrame(np.load('user_hist_new_df_fill0.npy'))
user_hist_new_df_fill0.columns = ['userId','movieId','rating','timestamp','userId2']+genres
user_hist_new_df_fill0 = user_hist_new_df_fill0.drop('userId2',1)

In [7]:
user_hist_new_ar_fill0 = np.array(user_hist_new_df_fill0)

In [ ]:
user_hist_new_ar_fill0

In [13]:
sum(np.array(list(map(lambda x: len(x[x==0]),user_hist_new_ar_fill0)))>4)

6731

筛选出打分次数至少为20次的用户记录

In [8]:
ratings_id_group = ratings.groupby('userId')
# 打分次数多于20次
user_rating_count = ratings_id_group['movieId'].count()
# 获取用户id
user20_ids = user_rating_count[user_rating_count >= 20].index
# 获取前10000个
user20_ids1 = user20_ids[0:500]

In [ ]:
import multiprocessing
# 并行计算 不要运行！！！
p = multiprocessing.Pool(20)
temp_re=p.map(user_genre_hist_new, user20_ids1)
p.close()
p.join()

In [21]:
user20_hist_new_df = pd.DataFrame(temp_re)
user20_hist_new_df.columns = ['userId','movieId','rating','timestamp','userId2']+genres

In [23]:
user20_hist_new_df = user20_hist_new_df.drop('userId2',1)
np.save('user20_hist_new_df',user20_hist_new_df)

In [24]:
user20_hist_new_df = pd.DataFrame(np.load('user20_hist_new_df.npy'))
user20_hist_new_df.columns = ['userId','movieId','rating','timestamp']+genres
user20_hist_new_df

,userId,movieId,rating,timestamp,Animation,History,Fantasy,Crime,Comedy,TV Movie,...,Mystery,Music,Adventure,Drama,Foreign,Thriller,Horror,Science Fiction,Documentary,Romance
0,1.0,73017.0,5.0,1.425943e+09,NaN,NaN,NaN,4.500000,4.250000,NaN,...,1.000000,NaN,NaN,4.125000,3.500000,NaN,NaN,NaN,5.000000,3.333333
1,2.0,339.0,5.0,8.670413e+08,NaN,3.500000,3.250000,3.000000,2.250000,NaN,...,3.500000,1.0,3.500,2.777778,NaN,4.000000,4.000000,4.000000,NaN,4.000000
2,4.0,2338.0,2.0,1.042675e+09,NaN,NaN,4.000000,4.250000,3.000000,NaN,...,3.500000,NaN,4.000,3.727273,1.000000,3.666667,4.000000,4.000000,4.000000,3.166667
3,5.0,2163.0,2.0,9.494245e+08,NaN,NaN,NaN,5.000000,5.000000,NaN,...,5.000000,NaN,2.000,5.000000,NaN,3.666667,5.000000,3.666667,NaN,5.000000
4,7.0,2858.0,4.5,1.486254e+09,NaN,3.000000,NaN,2.500000,4.000000,3.0,...,NaN,2.0,3.000,3.700000,2.500000,3.833333,2.500000,2.500000,2.166667,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16175.0,2355.0,4.0,1.457484e+09,4.0,4.333333,4.375000,3.673913,3.136364,3.5,...,4.250000,4.0,4.500,3.811321,4.166667,3.740741,3.833333,4.125000,3.600000,3.593750
9996,16177.0,166528.0,4.0,1.487639e+09,NaN,3.500000,NaN,5.000000,3.166667,0.5,...,4.500000,NaN,0.500,4.333333,NaN,4.250000,NaN,NaN,NaN,4.500000
9997,16179.0,273.0,1.0,8.303544e+08,NaN,2.000000,2.500000,3.571429,3.333333,NaN,...,3.000000,NaN,2.800,3.500000,NaN,3.600000,3.000000,2.800000,NaN,3.000000
9998,16181.0,97304.0,2.5,1.377809e+09,4.5,4.500000,4.166667,3.772727,4.083333,NaN,...,3.722222,3.0,3.625,4.071429,NaN,3.807692,2.000000,3.200000,3.000000,4.250000


In [26]:
# 缺失值用0插补
user20_hist_new_df_fill0 = user20_hist_new_df.fillna(value=0)
np.save('user20_hist_new_df_fill0',use20_hist_new_df_fill0)

In [27]:
user20_hist_new_df_fill0

,userId,movieId,rating,timestamp,Animation,History,Fantasy,Crime,Comedy,TV Movie,...,Mystery,Music,Adventure,Drama,Foreign,Thriller,Horror,Science Fiction,Documentary,Romance
0,1.0,73017.0,5.0,1.425943e+09,0.0,0.000000,0.000000,4.500000,4.250000,0.0,...,1.000000,0.0,0.000,4.125000,3.500000,0.000000,0.000000,0.000000,5.000000,3.333333
1,2.0,339.0,5.0,8.670413e+08,0.0,3.500000,3.250000,3.000000,2.250000,0.0,...,3.500000,1.0,3.500,2.777778,0.000000,4.000000,4.000000,4.000000,0.000000,4.000000
2,4.0,2338.0,2.0,1.042675e+09,0.0,0.000000,4.000000,4.250000,3.000000,0.0,...,3.500000,0.0,4.000,3.727273,1.000000,3.666667,4.000000,4.000000,4.000000,3.166667
3,5.0,2163.0,2.0,9.494245e+08,0.0,0.000000,0.000000,5.000000,5.000000,0.0,...,5.000000,0.0,2.000,5.000000,0.000000,3.666667,5.000000,3.666667,0.000000,5.000000
4,7.0,2858.0,4.5,1.486254e+09,0.0,3.000000,0.000000,2.500000,4.000000,3.0,...,0.000000,2.0,3.000,3.700000,2.500000,3.833333,2.500000,2.500000,2.166667,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16175.0,2355.0,4.0,1.457484e+09,4.0,4.333333,4.375000,3.673913,3.136364,3.5,...,4.250000,4.0,4.500,3.811321,4.166667,3.740741,3.833333,4.125000,3.600000,3.593750
9996,16177.0,166528.0,4.0,1.487639e+09,0.0,3.500000,0.000000,5.000000,3.166667,0.5,...,4.500000,0.0,0.500,4.333333,0.000000,4.250000,0.000000,0.000000,0.000000,4.500000
9997,16179.0,273.0,1.0,8.303544e+08,0.0,2.000000,2.500000,3.571429,3.333333,0.0,...,3.000000,0.0,2.800,3.500000,0.000000,3.600000,3.000000,2.800000,0.000000,3.000000
9998,16181.0,97304.0,2.5,1.377809e+09,4.5,4.500000,4.166667,3.772727,4.083333,0.0,...,3.722222,3.0,3.625,4.071429,0.000000,3.807692,2.000000,3.200000,3.000000,4.250000
